In [8]:
# Use centrailzed training to compare with federated learning
epochs = 30
n_train_items = 12800

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np


In [10]:
torch.cuda.is_available()

True

In [11]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [12]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

train_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
# test_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(test_loader)
#         if i < n_train_items / args.batch_size
#     ]


In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [14]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(),3)
        optimizer.step()
        
        
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader) * args.batch_size,
                100. * batch_idx / len(train_dataloader), loss.item()))

In [18]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        
        
        # add gussian noise before testing
        for param in model.parameters():
            param.add_(torch.from_numpy(np.random.normal(0,0.05,param.size())).to(device))
        
        
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.batch_size),
        100. * correct / (len(test_loader)*args.batch_size)))

In [19]:
with torch.no_grad():
    for param in model.parameters():
        print(param.size())
        param.add_(torch.from_numpy(np.random.normal(0,0.5,param.size())).to(device))
# print(list(model.parameters())[0])


torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [20]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_dataloader, optimizer, epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  if __name__ == '__main__':


Train Epoch: 1 [0/12800 (0%)]	Loss: 2.299716
Train Epoch: 1 [1920/12800 (15%)]	Loss: 2.154342
Train Epoch: 1 [3840/12800 (30%)]	Loss: 1.877676
Train Epoch: 1 [5760/12800 (45%)]	Loss: 1.357273
Train Epoch: 1 [7680/12800 (60%)]	Loss: 0.905127
Train Epoch: 1 [9600/12800 (75%)]	Loss: 0.496682
Train Epoch: 1 [11520/12800 (90%)]	Loss: 0.718168

Test set: Average loss: 2.2897, Accuracy: 3838/10048 (38%)

Train Epoch: 2 [0/12800 (0%)]	Loss: 2.308139
Train Epoch: 2 [1920/12800 (15%)]	Loss: 0.564057
Train Epoch: 2 [3840/12800 (30%)]	Loss: 0.368379
Train Epoch: 2 [5760/12800 (45%)]	Loss: 0.315816
Train Epoch: 2 [7680/12800 (60%)]	Loss: 0.414582
Train Epoch: 2 [9600/12800 (75%)]	Loss: 0.359035
Train Epoch: 2 [11520/12800 (90%)]	Loss: 0.575282

Test set: Average loss: 10.6912, Accuracy: 1967/10048 (20%)

Train Epoch: 3 [0/12800 (0%)]	Loss: 10.663928
Train Epoch: 3 [1920/12800 (15%)]	Loss: 0.857765
Train Epoch: 3 [3840/12800 (30%)]	Loss: 0.547203
Train Epoch: 3 [5760/12800 (45%)]	Loss: 0.324117
Trai